In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import warnings
import time
import sklearn.preprocessing

pd.set_option("display.max_columns", None)
warnings.filterwarnings(action='ignore')

serviceKey='bwUUK6MEA6GnqDpo2ZS9ChN4VZ1nkUPXrryU80XxUcLqxB62pIGIPWXNdyzqgT20BHrw8zxWWk9rqiSQwTrbnQ=='

p7000 ={'serviceKey' :serviceKey,
             'routeId':'200000112','stationId':'228000703'}
class Bus():
    def __init__(self,params):
        self.params=params
        self.bus_time=[]
        
    def repet(self):
        url = 'http://apis.data.go.kr/6410000/busarrivalservice/getBusArrivalItem'
        response = requests.get(url, params=self.params)
        soup=BeautifulSoup(response.text,'xml')
        return soup

        
    def current_buses(self): 
        url = 'http://apis.data.go.kr/6410000/buslocationservice/getBusLocationList'
        mod_params ={'serviceKey' : self.params['serviceKey'],
                  'routeId':self.params['routeId']}

        response=requests.get(url,params=mod_params)
        soup=BeautifulSoup(response.text,'xml')
        locations=sorted([int(x.stationSeq.text) for x in soup.find_all('busLocationList')])
        return locations


def trim_data(df,FileName):
    df['시간'] = df['시간'].apply(pd.to_datetime)
    df['hour']=df['시간'].dt.hour
    df['minute']=df['시간'].dt.minute
    df['second']=df['시간'].dt.second
    df['day'] = df['시간'].dt.weekday 

    mb = sklearn.preprocessing.MultiLabelBinarizer()
    if isinstance(df['운행중인 다른 버스들의 위치'].iloc[0], str):
        df['운행중인 다른 버스들의 위치'] = df['운행중인 다른 버스들의 위치'].apply(eval)
    entity = mb.fit_transform(df['운행중인 다른 버스들의 위치']) 
    lsts=pd.DataFrame(entity,columns=mb.classes_)
    lsts.columns=[f'station{x}' for x in lsts.columns]
    df=pd.concat([df,lsts],axis=1,sort=False).reset_index(drop=True)


    df=df.drop(["운행중인 다른 버스들의 위치"],axis=1)
    
    batch=df.index[df.arrived==1]
    df=df.iloc[:batch[-1]+1,:]
    #7/12 -----------------------------------
    batch=df.index[df.arrived==1]
    temp=0
    for x in batch:
        for y in range(temp,x): 
            df['arrived'][y]=(df['시간'][x]-df['시간'][y]).seconds
            temp=x+1
    for e in batch:
        df['arrived'][e]=0
    # 7/12 -----------------------------------

    df = df.set_index('시간')
    df.to_csv(f'./{FileName}')        # 동일 폴더에 파일이 저장될 거임
    return df




past=4
pBus=Bus(p7000)
for x in range(720): # 시간 설정
    soup=pBus.repet()
    try:
        result=int(soup.resultCode.text)
        arrived=0
        if past!=result:
            if result==0:
                arrived=1
        pBus.bus_time.append([soup.queryTime.text,pBus.current_buses(),arrived])
        past=result
        print(x,"번째 시도: ",pBus.bus_time)
        time.sleep(60)
    except Exception as e:
        continue

df=pd.DataFrame(pBus.bus_time,columns=['시간','운행중인 다른 버스들의 위치','arrived'])

mod_df=trim_data(df,'7000_wed.csv') # 이름 설정
mod_df

0 번째 시도:  [['2022-08-03 11:26:20.341', [9, 18, 26, 33, 42, 49, 61], 0]]
1 번째 시도:  [['2022-08-03 11:26:20.341', [9, 18, 26, 33, 42, 49, 61], 0], ['2022-08-03 11:27:20.550', [9, 19, 26, 33, 43, 50, 62], 0]]
2 번째 시도:  [['2022-08-03 11:26:20.341', [9, 18, 26, 33, 42, 49, 61], 0], ['2022-08-03 11:27:20.550', [9, 19, 26, 33, 43, 50, 62], 0], ['2022-08-03 11:28:20.713', [10, 20, 26, 34, 44, 50, 63], 0]]
3 번째 시도:  [['2022-08-03 11:26:20.341', [9, 18, 26, 33, 42, 49, 61], 0], ['2022-08-03 11:27:20.550', [9, 19, 26, 33, 43, 50, 62], 0], ['2022-08-03 11:28:20.713', [10, 20, 26, 34, 44, 50, 63], 0], ['2022-08-03 11:29:22.281', [10, 21, 26, 34, 45, 51, 63], 0]]
4 번째 시도:  [['2022-08-03 11:26:20.341', [9, 18, 26, 33, 42, 49, 61], 0], ['2022-08-03 11:27:20.550', [9, 19, 26, 33, 43, 50, 62], 0], ['2022-08-03 11:28:20.713', [10, 20, 26, 34, 44, 50, 63], 0], ['2022-08-03 11:29:22.281', [10, 21, 26, 34, 45, 51, 63], 0], ['2022-08-03 11:30:23.443', [11, 21, 26, 36, 46, 51, 65], 0]]
5 번째 시도:  [['2022-08-03 

16 번째 시도:  [['2022-08-03 11:26:20.341', [9, 18, 26, 33, 42, 49, 61], 0], ['2022-08-03 11:27:20.550', [9, 19, 26, 33, 43, 50, 62], 0], ['2022-08-03 11:28:20.713', [10, 20, 26, 34, 44, 50, 63], 0], ['2022-08-03 11:29:22.281', [10, 21, 26, 34, 45, 51, 63], 0], ['2022-08-03 11:30:23.443', [11, 21, 26, 36, 46, 51, 65], 0], ['2022-08-03 11:31:23.605', [2, 13, 22, 27, 37, 46, 51, 65], 1], ['2022-08-03 11:32:25.033', [3, 13, 23, 27, 38, 47, 52, 65], 0], ['2022-08-03 11:33:25.211', [3, 14, 24, 27, 38, 48, 52, 67], 0], ['2022-08-03 11:34:25.357', [3, 15, 24, 28, 39, 48, 52, 68], 0], ['2022-08-03 11:35:26.066', [4, 15, 25, 28, 40, 48, 53, 68], 0], ['2022-08-03 11:36:26.291', [5, 16, 25, 29, 40, 49, 53, 69], 0], ['2022-08-03 11:37:26.453', [6, 16, 26, 29, 41, 50, 53, 69], 0], ['2022-08-03 11:38:26.630', [8, 16, 26, 30, 42, 50, 54, 70], 0], ['2022-08-03 11:39:26.823', [8, 17, 26, 31, 43, 51, 54, 70], 0], ['2022-08-03 11:40:27.157', [9, 18, 26, 31, 44, 51, 55, 72], 0], ['2022-08-03 11:41:27.584', [9

23 번째 시도:  [['2022-08-03 11:26:20.341', [9, 18, 26, 33, 42, 49, 61], 0], ['2022-08-03 11:27:20.550', [9, 19, 26, 33, 43, 50, 62], 0], ['2022-08-03 11:28:20.713', [10, 20, 26, 34, 44, 50, 63], 0], ['2022-08-03 11:29:22.281', [10, 21, 26, 34, 45, 51, 63], 0], ['2022-08-03 11:30:23.443', [11, 21, 26, 36, 46, 51, 65], 0], ['2022-08-03 11:31:23.605', [2, 13, 22, 27, 37, 46, 51, 65], 1], ['2022-08-03 11:32:25.033', [3, 13, 23, 27, 38, 47, 52, 65], 0], ['2022-08-03 11:33:25.211', [3, 14, 24, 27, 38, 48, 52, 67], 0], ['2022-08-03 11:34:25.357', [3, 15, 24, 28, 39, 48, 52, 68], 0], ['2022-08-03 11:35:26.066', [4, 15, 25, 28, 40, 48, 53, 68], 0], ['2022-08-03 11:36:26.291', [5, 16, 25, 29, 40, 49, 53, 69], 0], ['2022-08-03 11:37:26.453', [6, 16, 26, 29, 41, 50, 53, 69], 0], ['2022-08-03 11:38:26.630', [8, 16, 26, 30, 42, 50, 54, 70], 0], ['2022-08-03 11:39:26.823', [8, 17, 26, 31, 43, 51, 54, 70], 0], ['2022-08-03 11:40:27.157', [9, 18, 26, 31, 44, 51, 55, 72], 0], ['2022-08-03 11:41:27.584', [9

28 번째 시도:  [['2022-08-03 11:26:20.341', [9, 18, 26, 33, 42, 49, 61], 0], ['2022-08-03 11:27:20.550', [9, 19, 26, 33, 43, 50, 62], 0], ['2022-08-03 11:28:20.713', [10, 20, 26, 34, 44, 50, 63], 0], ['2022-08-03 11:29:22.281', [10, 21, 26, 34, 45, 51, 63], 0], ['2022-08-03 11:30:23.443', [11, 21, 26, 36, 46, 51, 65], 0], ['2022-08-03 11:31:23.605', [2, 13, 22, 27, 37, 46, 51, 65], 1], ['2022-08-03 11:32:25.033', [3, 13, 23, 27, 38, 47, 52, 65], 0], ['2022-08-03 11:33:25.211', [3, 14, 24, 27, 38, 48, 52, 67], 0], ['2022-08-03 11:34:25.357', [3, 15, 24, 28, 39, 48, 52, 68], 0], ['2022-08-03 11:35:26.066', [4, 15, 25, 28, 40, 48, 53, 68], 0], ['2022-08-03 11:36:26.291', [5, 16, 25, 29, 40, 49, 53, 69], 0], ['2022-08-03 11:37:26.453', [6, 16, 26, 29, 41, 50, 53, 69], 0], ['2022-08-03 11:38:26.630', [8, 16, 26, 30, 42, 50, 54, 70], 0], ['2022-08-03 11:39:26.823', [8, 17, 26, 31, 43, 51, 54, 70], 0], ['2022-08-03 11:40:27.157', [9, 18, 26, 31, 44, 51, 55, 72], 0], ['2022-08-03 11:41:27.584', [9

33 번째 시도:  [['2022-08-03 11:26:20.341', [9, 18, 26, 33, 42, 49, 61], 0], ['2022-08-03 11:27:20.550', [9, 19, 26, 33, 43, 50, 62], 0], ['2022-08-03 11:28:20.713', [10, 20, 26, 34, 44, 50, 63], 0], ['2022-08-03 11:29:22.281', [10, 21, 26, 34, 45, 51, 63], 0], ['2022-08-03 11:30:23.443', [11, 21, 26, 36, 46, 51, 65], 0], ['2022-08-03 11:31:23.605', [2, 13, 22, 27, 37, 46, 51, 65], 1], ['2022-08-03 11:32:25.033', [3, 13, 23, 27, 38, 47, 52, 65], 0], ['2022-08-03 11:33:25.211', [3, 14, 24, 27, 38, 48, 52, 67], 0], ['2022-08-03 11:34:25.357', [3, 15, 24, 28, 39, 48, 52, 68], 0], ['2022-08-03 11:35:26.066', [4, 15, 25, 28, 40, 48, 53, 68], 0], ['2022-08-03 11:36:26.291', [5, 16, 25, 29, 40, 49, 53, 69], 0], ['2022-08-03 11:37:26.453', [6, 16, 26, 29, 41, 50, 53, 69], 0], ['2022-08-03 11:38:26.630', [8, 16, 26, 30, 42, 50, 54, 70], 0], ['2022-08-03 11:39:26.823', [8, 17, 26, 31, 43, 51, 54, 70], 0], ['2022-08-03 11:40:27.157', [9, 18, 26, 31, 44, 51, 55, 72], 0], ['2022-08-03 11:41:27.584', [9

...

182 번째 시도:  [['2022-08-03 11:26:20.341', [9, 18, 26, 33, 42, 49, 61], 0], ['2022-08-03 11:27:20.550', [9, 19, 26, 33, 43, 50, 62], 0], ['2022-08-03 11:28:20.713', [10, 20, 26, 34, 44, 50, 63], 0], ['2022-08-03 11:29:22.281', [10, 21, 26, 34, 45, 51, 63], 0], ['2022-08-03 11:30:23.443', [11, 21, 26, 36, 46, 51, 65], 0], ['2022-08-03 11:31:23.605', [2, 13, 22, 27, 37, 46, 51, 65], 1], ['2022-08-03 11:32:25.033', [3, 13, 23, 27, 38, 47, 52, 65], 0], ['2022-08-03 11:33:25.211', [3, 14, 24, 27, 38, 48, 52, 67], 0], ['2022-08-03 11:34:25.357', [3, 15, 24, 28, 39, 48, 52, 68], 0], ['2022-08-03 11:35:26.066', [4, 15, 25, 28, 40, 48, 53, 68], 0], ['2022-08-03 11:36:26.291', [5, 16, 25, 29, 40, 49, 53, 69], 0], ['2022-08-03 11:37:26.453', [6, 16, 26, 29, 41, 50, 53, 69], 0], ['2022-08-03 11:38:26.630', [8, 16, 26, 30, 42, 50, 54, 70], 0], ['2022-08-03 11:39:26.823', [8, 17, 26, 31, 43, 51, 54, 70], 0], ['2022-08-03 11:40:27.157', [9, 18, 26, 31, 44, 51, 55, 72], 0], ['2022-08-03 11:41:27.584', [

183 번째 시도:  [['2022-08-03 11:26:20.341', [9, 18, 26, 33, 42, 49, 61], 0], ['2022-08-03 11:27:20.550', [9, 19, 26, 33, 43, 50, 62], 0], ['2022-08-03 11:28:20.713', [10, 20, 26, 34, 44, 50, 63], 0], ['2022-08-03 11:29:22.281', [10, 21, 26, 34, 45, 51, 63], 0], ['2022-08-03 11:30:23.443', [11, 21, 26, 36, 46, 51, 65], 0], ['2022-08-03 11:31:23.605', [2, 13, 22, 27, 37, 46, 51, 65], 1], ['2022-08-03 11:32:25.033', [3, 13, 23, 27, 38, 47, 52, 65], 0], ['2022-08-03 11:33:25.211', [3, 14, 24, 27, 38, 48, 52, 67], 0], ['2022-08-03 11:34:25.357', [3, 15, 24, 28, 39, 48, 52, 68], 0], ['2022-08-03 11:35:26.066', [4, 15, 25, 28, 40, 48, 53, 68], 0], ['2022-08-03 11:36:26.291', [5, 16, 25, 29, 40, 49, 53, 69], 0], ['2022-08-03 11:37:26.453', [6, 16, 26, 29, 41, 50, 53, 69], 0], ['2022-08-03 11:38:26.630', [8, 16, 26, 30, 42, 50, 54, 70], 0], ['2022-08-03 11:39:26.823', [8, 17, 26, 31, 43, 51, 54, 70], 0], ['2022-08-03 11:40:27.157', [9, 18, 26, 31, 44, 51, 55, 72], 0], ['2022-08-03 11:41:27.584', [

184 번째 시도:  [['2022-08-03 11:26:20.341', [9, 18, 26, 33, 42, 49, 61], 0], ['2022-08-03 11:27:20.550', [9, 19, 26, 33, 43, 50, 62], 0], ['2022-08-03 11:28:20.713', [10, 20, 26, 34, 44, 50, 63], 0], ['2022-08-03 11:29:22.281', [10, 21, 26, 34, 45, 51, 63], 0], ['2022-08-03 11:30:23.443', [11, 21, 26, 36, 46, 51, 65], 0], ['2022-08-03 11:31:23.605', [2, 13, 22, 27, 37, 46, 51, 65], 1], ['2022-08-03 11:32:25.033', [3, 13, 23, 27, 38, 47, 52, 65], 0], ['2022-08-03 11:33:25.211', [3, 14, 24, 27, 38, 48, 52, 67], 0], ['2022-08-03 11:34:25.357', [3, 15, 24, 28, 39, 48, 52, 68], 0], ['2022-08-03 11:35:26.066', [4, 15, 25, 28, 40, 48, 53, 68], 0], ['2022-08-03 11:36:26.291', [5, 16, 25, 29, 40, 49, 53, 69], 0], ['2022-08-03 11:37:26.453', [6, 16, 26, 29, 41, 50, 53, 69], 0], ['2022-08-03 11:38:26.630', [8, 16, 26, 30, 42, 50, 54, 70], 0], ['2022-08-03 11:39:26.823', [8, 17, 26, 31, 43, 51, 54, 70], 0], ['2022-08-03 11:40:27.157', [9, 18, 26, 31, 44, 51, 55, 72], 0], ['2022-08-03 11:41:27.584', [

185 번째 시도:  [['2022-08-03 11:26:20.341', [9, 18, 26, 33, 42, 49, 61], 0], ['2022-08-03 11:27:20.550', [9, 19, 26, 33, 43, 50, 62], 0], ['2022-08-03 11:28:20.713', [10, 20, 26, 34, 44, 50, 63], 0], ['2022-08-03 11:29:22.281', [10, 21, 26, 34, 45, 51, 63], 0], ['2022-08-03 11:30:23.443', [11, 21, 26, 36, 46, 51, 65], 0], ['2022-08-03 11:31:23.605', [2, 13, 22, 27, 37, 46, 51, 65], 1], ['2022-08-03 11:32:25.033', [3, 13, 23, 27, 38, 47, 52, 65], 0], ['2022-08-03 11:33:25.211', [3, 14, 24, 27, 38, 48, 52, 67], 0], ['2022-08-03 11:34:25.357', [3, 15, 24, 28, 39, 48, 52, 68], 0], ['2022-08-03 11:35:26.066', [4, 15, 25, 28, 40, 48, 53, 68], 0], ['2022-08-03 11:36:26.291', [5, 16, 25, 29, 40, 49, 53, 69], 0], ['2022-08-03 11:37:26.453', [6, 16, 26, 29, 41, 50, 53, 69], 0], ['2022-08-03 11:38:26.630', [8, 16, 26, 30, 42, 50, 54, 70], 0], ['2022-08-03 11:39:26.823', [8, 17, 26, 31, 43, 51, 54, 70], 0], ['2022-08-03 11:40:27.157', [9, 18, 26, 31, 44, 51, 55, 72], 0], ['2022-08-03 11:41:27.584', [

KeyboardInterrupt: 